# Italian Wine Recommender - Static Version

## I've gathered a database of over 120,000 wines from all around the world. Among them, I've filtered about 5,000 top-notch Italian wines that have scored between 90 and 100. The goal is to help wine enthusiasts everywhere find the perfect Italian wine to complement their meals. The Italian Wine recommender gives the top 10 most similar wines in the dataset.

### I've chosen to present this static version of the project as a preview while I continue developing the interactive web application(Integration). In the upcoming Web App, users will have the ability to engage by requesting wine recommendations based on their preferences, and the app will provide tailored recommendations accordingly.

In [122]:
!pip --version


pip 22.2.2 from C:\Users\jefff\anaconda3\lib\site-packages\pip (python 3.9)



In [125]:
!pip install langdetect
!pip install googletrans==4.0.0-rc1

In [274]:
!pip install mtranslate

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3673 sha256=8bfc393f52a0c4c67cd44d2d9f0dea3a26ff2b1d07546c630d4862b305175b58
  Stored in directory: c:\users\jefff\appdata\local\pip\cache\wheels\a2\20\13\93c62f314d4a29db25b1b24b2c38f79eb1beb558c727b894a3
Successfully built mtranslate


In [126]:
# Import pandas for data manipulation and analysis
import pandas as pd  
# Import TfidfVectorizer for text-based feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer  
# Import linear_kernel for calculating cosine similarities
from sklearn.metrics.pairwise import linear_kernel  
# Import detect from langdetect for language detection
from langdetect import detect  
 # Import Translator from googletrans for language translation
from googletrans import Translator 

In [259]:
# Load your wine dataset into a DataFrame (replace 'wine_data.csv' with your dataset)
wine_data = pd.read_csv('C:/Users/jefff/Desktop/Python_Wine.Review_Analysis/winemag-data-130k-v2.csv')

In [260]:
wine_data

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


## Data Cleaning

In [261]:
# Rows with null("NaN") values across 'country' column ONLY 
wine_data[wine_data['country'].isna()]

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
913,913,NaN,"Amber in color, this wine has aromas of peach ...",Asureti Valley,87,30.0,NaN,NaN,NaN,Mike DeSimone,@worldwineguys,Gotsa Family Wines 2014 Asureti Valley Chinuri,Chinuri,Gotsa Family Wines
3131,3131,NaN,"Soft, fruity and juicy, this is a pleasant, si...",Partager,83,NaN,NaN,NaN,NaN,Roger Voss,@vossroger,Barton & Guestier NV Partager Red,Red Blend,Barton & Guestier
4243,4243,NaN,"Violet-red in color, this semisweet wine has a...",Red Naturally Semi-Sweet,88,18.0,NaN,NaN,NaN,Mike DeSimone,@worldwineguys,Kakhetia Traditional Winemaking 2012 Red Natur...,Ojaleshi,Kakhetia Traditional Winemaking
9509,9509,NaN,This mouthwatering blend starts with a nose of...,Theopetra Malagouzia-Assyrtiko,92,28.0,NaN,NaN,NaN,Susan Kostrzewa,@suskostrzewa,Tsililis 2015 Theopetra Malagouzia-Assyrtiko W...,White Blend,Tsililis
9750,9750,NaN,This orange-style wine has a cloudy yellow-gol...,Orange Nikolaevo Vineyard,89,28.0,NaN,NaN,NaN,Jeff Jenssen,@worldwineguys,Ross-idi 2015 Orange Nikolaevo Vineyard Chardo...,Chardonnay,Ross-idi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124176,124176,NaN,This Swiss red blend is composed of four varie...,Les Romaines,90,30.0,NaN,NaN,NaN,Jeff Jenssen,@worldwineguys,Les Frères Dutruy 2014 Les Romaines Red,Red Blend,Les Frères Dutruy
129407,129407,NaN,Dry spicy aromas of dusty plum and tomato add ...,Reserve,89,22.0,NaN,NaN,NaN,Michael Schachner,@wineschach,El Capricho 2015 Reserve Cabernet Sauvignon,Cabernet Sauvignon,El Capricho
129408,129408,NaN,El Capricho is one of Uruguay's more consisten...,Reserve,89,22.0,NaN,NaN,NaN,Michael Schachner,@wineschach,El Capricho 2015 Reserve Tempranillo,Tempranillo,El Capricho
129590,129590,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Mike DeSimone,@worldwineguys,Büyülübağ 2012 Shah Red,Red Blend,Büyülübağ


In [262]:
# Rows with null("NaN") values across ALL columns 
wine_data[wine_data.isna().any(axis=1)]

# Null("NaN") values exist in multiple columns, not limited to just the "country" column.

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [246]:
# Create a new DataFrame 'rows_with_nan_price' containing rows where the 'price' column is 'NaN'
rows_with_nan_price = wine_data[wine_data['price'].isna()]

# Display the 'rows_with_nan_price' DataFrame, showing rows with missing 'price' values
rows_with_nan_price



,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
13,13,Italy,This is dominated by oak and oak-driven aromas...,Rosso,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte
30,30,France,Red cherry fruit comes laced with light tannin...,Nouveau,86,NaN,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Domaine de la Madone 2012 Nouveau (Beaujolais...,Gamay,Domaine de la Madone
31,31,Italy,Merlot and Nero d'Avola form the base for this...,Calanìca Nero d'Avola-Merlot,86,NaN,Sicily & Sardinia,Sicilia,NaN,NaN,NaN,Duca di Salaparuta 2010 Calanìca Nero d'Avola-...,Red Blend,Duca di Salaparuta
32,32,Italy,"Part of the extended Calanìca series, this Gri...",Calanìca Grillo-Viognier,86,NaN,Sicily & Sardinia,Sicilia,NaN,NaN,NaN,Duca di Salaparuta 2011 Calanìca Grillo-Viogni...,White Blend,Duca di Salaparuta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129844,129844,Italy,"Doga delle Clavule is a neutral, mineral-drive...",Doga delle Clavule,86,NaN,Tuscany,Morellino di Scansano,NaN,NaN,NaN,Caparzo 2006 Doga delle Clavule (Morellino di...,Sangiovese,Caparzo
129860,129860,Portugal,This rich wine has a firm structure as well as...,Pacheca Superior,90,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta da Pacheca 2013 Pacheca Superior Red (D...,Portuguese Red,Quinta da Pacheca
129863,129863,Portugal,This mature wine that has 50% Touriga Nacional...,Reserva,90,NaN,Dão,NaN,NaN,Roger Voss,@vossroger,Seacampo 2011 Reserva Red (Dão),Portuguese Red,Seacampo
129893,129893,Italy,"Aromas of passion fruit, hay and a vegetal not...",Corte Menini,91,NaN,Veneto,Soave Classico,NaN,Kerin O’Keefe,@kerinokeefe,Le Mandolare 2015 Corte Menini (Soave Classico),Garganega,Le Mandolare


In [265]:
# List of column names where 'NaN' values should be replaced with 'unknown' except column 'price' and 'points'
columns_to_fillna = ['country', 'description', 'designation', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety', 'winery']

# Replace 'NaN' values with 'unknown' in the specified columns
wine_data[columns_to_fillna] = wine_data[columns_to_fillna].fillna('unknown')



In [266]:
# Checking if the data was cleaned after replacing all nuLL("NAN") values in the DataFrame 'data' with the string 'unknown' in place
wine_data[wine_data.isna().any(axis=1)]

# No more null ('NaN') values found, except in the 'points' and 'price' column.

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
13,13,Italy,This is dominated by oak and oak-driven aromas...,Rosso,87,NaN,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte
30,30,France,Red cherry fruit comes laced with light tannin...,Nouveau,86,NaN,Beaujolais,Beaujolais-Villages,unknown,Roger Voss,@vossroger,Domaine de la Madone 2012 Nouveau (Beaujolais...,Gamay,Domaine de la Madone
31,31,Italy,Merlot and Nero d'Avola form the base for this...,Calanìca Nero d'Avola-Merlot,86,NaN,Sicily & Sardinia,Sicilia,unknown,unknown,unknown,Duca di Salaparuta 2010 Calanìca Nero d'Avola-...,Red Blend,Duca di Salaparuta
32,32,Italy,"Part of the extended Calanìca series, this Gri...",Calanìca Grillo-Viognier,86,NaN,Sicily & Sardinia,Sicilia,unknown,unknown,unknown,Duca di Salaparuta 2011 Calanìca Grillo-Viogni...,White Blend,Duca di Salaparuta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129844,129844,Italy,"Doga delle Clavule is a neutral, mineral-drive...",Doga delle Clavule,86,NaN,Tuscany,Morellino di Scansano,unknown,unknown,unknown,Caparzo 2006 Doga delle Clavule (Morellino di...,Sangiovese,Caparzo
129860,129860,Portugal,This rich wine has a firm structure as well as...,Pacheca Superior,90,NaN,Douro,unknown,unknown,Roger Voss,@vossroger,Quinta da Pacheca 2013 Pacheca Superior Red (D...,Portuguese Red,Quinta da Pacheca
129863,129863,Portugal,This mature wine that has 50% Touriga Nacional...,Reserva,90,NaN,Dão,unknown,unknown,Roger Voss,@vossroger,Seacampo 2011 Reserva Red (Dão),Portuguese Red,Seacampo
129893,129893,Italy,"Aromas of passion fruit, hay and a vegetal not...",Corte Menini,91,NaN,Veneto,Soave Classico,unknown,Kerin O’Keefe,@kerinokeefe,Le Mandolare 2015 Corte Menini (Soave Classico),Garganega,Le Mandolare


## Exploratory Analysis

In [267]:
wine_data

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,unknown,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,unknown,unknown,Anna Lee C. Iijima,unknown,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,unknown,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,unknown,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",unknown,90,32.0,Alsace,Alsace,unknown,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


## All Italian Wines

In [268]:
# Filter the DataFrame to select Italian wines
italy_wines = wine_data[wine_data['country'] == 'Italy']

italy_wines

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
13,13,Italy,This is dominated by oak and oak-driven aromas...,Rosso,87,NaN,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte
22,22,Italy,Delicate aromas recall white flower and citrus...,Ficiligno,87,19.0,Sicily & Sardinia,Sicilia,unknown,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto
24,24,Italy,"Aromas of prune, blackcurrant, toast and oak c...",Aynat,87,35.0,Sicily & Sardinia,Sicilia,unknown,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129929,129929,Italy,"This luminous sparkler has a sweet, fruit-forw...",unknown,91,38.0,Veneto,Prosecco Superiore di Cartizze,unknown,unknown,unknown,Col Vetoraz Spumanti NV Prosecco Superiore di...,Prosecco,Col Vetoraz Spumanti
129943,129943,Italy,"A blend of Nero d'Avola and Syrah, this convey...",Adènzia,90,29.0,Sicily & Sardinia,Sicilia,unknown,Kerin O’Keefe,@kerinokeefe,Baglio del Cristo di Campobello 2012 Adènzia R...,Red Blend,Baglio del Cristo di Campobello
129947,129947,Italy,"A blend of 65% Cabernet Sauvignon, 30% Merlot ...",Symposio,90,20.0,Sicily & Sardinia,Terre Siciliane,unknown,Kerin O’Keefe,@kerinokeefe,Feudo Principi di Butera 2012 Symposio Red (Te...,Red Blend,Feudo Principi di Butera
129961,129961,Italy,"Intense aromas of wild cherry, baking spice, t...",unknown,90,30.0,Sicily & Sardinia,Sicilia,unknown,Kerin O’Keefe,@kerinokeefe,COS 2013 Frappato (Sicilia),Frappato,COS


## ~ 5.000 Italian Wines ( Points > 90)

In [271]:
# Filter Italian wines for points above 90 and sort by points in descending order
italy_wines_rec = italy_wines[italy_wines['points'] > 90].sort_values(by='points', ascending=False).head(5000)


italy_wines_rec

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
45781,45781,Italy,"This gorgeous, fragrant wine opens with classi...",Riserva,100,550.0,Tuscany,Brunello di Montalcino,unknown,Kerin O’Keefe,@kerinokeefe,Biondi Santi 2010 Riserva (Brunello di Montal...,Sangiovese,Biondi Santi
39286,39286,Italy,"A perfect wine from a classic vintage, the 200...",Masseto,100,460.0,Tuscany,Toscana,unknown,unknown,unknown,Tenuta dell'Ornellaia 2007 Masseto Merlot (Tos...,Merlot,Tenuta dell'Ornellaia
111754,111754,Italy,It takes only a few moments before you appreci...,Cerretalto,100,270.0,Tuscany,Brunello di Montalcino,unknown,unknown,unknown,Casanova di Neri 2007 Cerretalto (Brunello di...,Sangiovese Grosso,Casanova di Neri
7335,7335,Italy,Thick as molasses and dark as caramelized brow...,Occhio di Pernice,100,210.0,Tuscany,Vin Santo di Montepulciano,unknown,unknown,unknown,Avignonesi 1995 Occhio di Pernice (Vin Santo ...,Prugnolo Gentile,Avignonesi
79104,79104,Italy,Even better than the highly acclaimed 2001 vin...,Masseto,99,250.0,Tuscany,Toscana,unknown,unknown,unknown,Tenuta dell'Ornellaia 2004 Masseto Merlot (Tos...,Merlot,Tenuta dell'Ornellaia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41962,41962,Italy,Vittorio Moretti Extra Brut is an elegant spar...,Vittorio Moretti Extra Brut,91,NaN,Lombardy,Franciacorta,unknown,unknown,unknown,Bellavista 2002 Vittorio Moretti Extra Brut Sp...,Sparkling Blend,Bellavista
41965,41965,Italy,Ferrari's Brut Perlé (made with 100% Chardonna...,Brut Perlé,91,32.0,Northeastern Italy,Trento,unknown,unknown,unknown,Ferrari 2004 Brut Perlé Chardonnay (Trento),Chardonnay,Ferrari
41969,41969,Italy,This wine is a result of careful winemaking in...,Satèn Millesimato Brut,91,36.0,Lombardy,Franciacorta,unknown,unknown,unknown,La Valle 2004 Satèn Millesimato Brut Chardonna...,Chardonnay,La Valle
41970,41970,Italy,This is an intense and austere sparkling wine ...,Brut Riserva,91,60.0,Northeastern Italy,Trento,unknown,unknown,unknown,Methius 2004 Brut Riserva Sparkling (Trento),Sparkling Blend,Methius


In [270]:
# Display the dimensions (number of rows and columns) of the DataFrame 'data'
italy_wines_rec.shape

(4265, 14)

# Italian Wine Recommendation System
## Gives the top 10 most similar wines in the dataset.
### The code  - Encapsulates the entire functionality into the "recommend_wines" function and call it by typing recommend_wines()

In [ ]:

def recommend_wines():
    # Load your wine dataset into a DataFrame (replace 'wine_data.csv' with your dataset)
    wine_data = pd.read_csv('C:/Users/jefff/Desktop/Python_Wine.Review_Analysis/winemag-data-130k-v2.csv')

    # Filter the DataFrame to select Italian wines
    italy_wines = wine_data[wine_data['country'] == 'Italy']

    # Filter Italian wines for points above 90 and sort by points in descending order
    italy_wines_rec = italy_wines[italy_wines['points'] > 90].sort_values(by='points', ascending=False).head(5000)

    # Function to translate Italian queries to English
    def translate_to_english(query):
        translator = Translator()
        translated = translator.translate(query, src='it', dest='en')
        return translated.text

    # User Interaction: Input a query
    print("Select language for input:")
    print("1. English")
    print("2. Italian")
    language_choice = input("Enter 1 or 2: ")

    if language_choice == '1':
        user_query = input("Enter your wine recommendation query in English: ")
    elif language_choice == '2':
        user_query = input("Enter your wine recommendation query in Italian: ")
        # Translate Italian query to English
        user_query = translate_to_english(user_query)
    else:
        print("Invalid choice. Please select 1 or 2.")
        return

    # Combine relevant text columns into a single 'combined_text' column for Italian wines above 90 points
    text_columns = ['country', 'description', 'title', 'variety', 'winery']
    italy_wines_rec['combined_text'] = italy_wines_rec[text_columns].apply(lambda x: ' '.join(map(str, x)), axis=1)

    # TF-IDF Vectorization for Italian wines above 90 points
    # TF-IDF stands for "Term Frequency-Inverse Document Frequency,".
    #it is a technique used in natural language processing and information retrieval to convert a collection of text documents into a numerical format that can be used for various machine learning tasks.
    # TF-IDF vectorization is primarily used for text data analysis, document clustering, and information retrieval.
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(italy_wines_rec['combined_text'])

    # Compute cosine similarity between wines for Italian wines above 90 points
    cosine_sim_italy_wines_rec = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Function to get wine recommendations based on user query for Italian wines above 90 points
    def get_recommendations_italy_wines_rec(query, cosine_sim=cosine_sim_italy_wines_rec):
        # Create a Series with wine indices and cosine similarity scores
        wine_indices = pd.Series(italy_wines_rec.index, index=italy_wines_rec['title'])
        wine_scores = {}

        # Tokenize and transform the user query using the TF-IDF vectorizer
        query_vector = tfidf_vectorizer.transform([query])

        # Calculate cosine similarity between user query and all wines in the dataset
        cosine_scores = linear_kernel(query_vector, tfidf_matrix).flatten()

        # Get the top 10 most similar wines in the dataset
        top_indices = cosine_scores.argsort()[:-11:-1]

        for idx in top_indices:
            wine_scores[italy_wines_rec['title'].iloc[idx]] = cosine_scores[idx]

        return wine_scores

    # Get recommendations based on the processed query
    recommendations = get_recommendations_italy_wines_rec(user_query)

    # Print the results
    print("\n\n********** Recommended Wines **********\n")
    if recommendations:
        for wine, score in recommendations.items():
            top_wine_info = italy_wines_rec.loc[italy_wines_rec['title'] == wine].iloc[0]
            print(f"Country: {top_wine_info['country']}")
            print(f"Score: {top_wine_info['points']}/100")
            print(f"Price: {top_wine_info['price']}")
            print(f"Wine: {wine}")
            print(f"Province: {top_wine_info['province']}")
            print(f"Variety: {top_wine_info['variety']}")
            print(f"Winery: {top_wine_info['winery']}")
            print(f"Statistical Match: {score * 100:.2f}%")
            print("---------------------------------------------------------------------------------")
    else:
        print("No recommendations found for the given query.")

# Call the function to run the wine recommendation system
recommend_wines()


###   Function "recommend_wines()"

In [ ]:
recommend_wines()

## Input language: English

### Which is the best wine to drink with pasta?

In [377]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 1
Enter your wine recommendation query in English: Which is the best wine to drink with pasta?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: nan
Wine: Bolsignano 2008 Grazia  (Brunello di Montalcino)
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Bolsignano
Statistical Match: 17.51%
---------------------------------------------------------------------------------
Country: Italy
Score: 92/100
Price: nan
Wine: Tormaresca 2007 Torcicoda Primitivo (Salento)
Province: Southern Italy
Variety: Primitivo
Winery: Tormaresca
Statistical Match: 17.48%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: 31.0
Wine: Castello Banfi 2009 Cum Laude Red (Toscana)
Province: Tuscany
Variety: Red Blend
Winery: Castello Banfi
Statistical Match: 17.46%
---------------------------------------------------------------------------------
Country: I

### Which wine would you recommend to drink with meat?

In [388]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 1
Enter your wine recommendation query in English: Which wine would you recommend to drink with meat?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: 30.0
Wine: Marchesi Antinori 2010 Guado al Tasso Il Bruciato  (Bolgheri)
Province: Tuscany
Variety: Red Blend
Winery: Marchesi Antinori
Statistical Match: 22.88%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: 26.0
Wine: Renieri 2007 Riserva  (Chianti Classico)
Province: Tuscany
Variety: Sangiovese
Winery: Renieri
Statistical Match: 20.57%
---------------------------------------------------------------------------------
Country: Italy
Score: 92/100
Price: 80.0
Wine: Talenti 2007 Pian di Conte Riserva  (Brunello di Montalcino)
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Talenti
Statistical Match: 19.83%
-------------------------------------------------------------------

### Which is the best wine to drink with fish?

In [379]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 1
Enter your wine recommendation query in English: Which is the best wine to drink with fish?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: nan
Wine: Bellavista 2002 Vittorio Moretti Extra Brut Sparkling (Franciacorta)
Province: Lombardy
Variety: Sparkling Blend
Winery: Bellavista
Statistical Match: 17.95%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 80.0
Wine: Speri 2008 Vigneto Monte Sant'Urbano  (Amarone della Valpolicella Classico)
Province: Veneto
Variety: Corvina, Rondinella, Molinara
Winery: Speri
Statistical Match: 10.64%
---------------------------------------------------------------------------------
Country: Italy
Score: 93/100
Price: 64.0
Wine: Terrabianca 2004 Ceppate Cabernet Sauvignon (Toscana)
Province: Tuscany
Variety: Cabernet Sauvignon
Winery: Terrabianca
Statistical Match: 9.53%
---------------------

### Which is the best wine to drink with seafood?

In [380]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 1
Enter your wine recommendation query in English: Which is the best wine to drink with seafood?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: 55.0
Wine: Cantina Terlano 2008 Quarz Sauvignon (Alto Adige)
Province: Northeastern Italy
Variety: Sauvignon
Winery: Cantina Terlano
Statistical Match: 19.52%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: nan
Wine: Ronchi di Manzano 1999 Superiore Tocai (Colli Orientali del Friuli)
Province: Northeastern Italy
Variety: Tocai
Winery: Ronchi di Manzano
Statistical Match: 12.84%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 80.0
Wine: Speri 2008 Vigneto Monte Sant'Urbano  (Amarone della Valpolicella Classico)
Province: Veneto
Variety: Corvina, Rondinella, Molinara
Winery: Speri
Statistical Match: 11.01%
---------

### Which wine would be the most suitable for veggies?

In [390]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 1
Enter your wine recommendation query in English: Which wine would be the most suitable for veggies?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: nan
Wine: Mario Gagliasso 2004 Riserva  (Barolo)
Province: Piedmont
Variety: Nebbiolo
Winery: Mario Gagliasso
Statistical Match: 27.85%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 55.0
Wine: Verbena 2007  Brunello di Montalcino
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Verbena
Statistical Match: 4.28%
---------------------------------------------------------------------------------
Country: Italy
Score: 93/100
Price: 55.0
Wine: Caprili 2004 Riserva  (Brunello di Montalcino)
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Caprili
Statistical Match: 4.13%
---------------------------------------------------------------------------------
Country: Italy
Score: 92

## Input language: Italian

### Quale è il miglior vino da bere con la pasta?

In [381]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 2
Enter your wine recommendation query in Italian: Quale è il miglior vino da bere con la pasta?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: nan
Wine: Bolsignano 2008 Grazia  (Brunello di Montalcino)
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Bolsignano
Statistical Match: 17.51%
---------------------------------------------------------------------------------
Country: Italy
Score: 92/100
Price: nan
Wine: Tormaresca 2007 Torcicoda Primitivo (Salento)
Province: Southern Italy
Variety: Primitivo
Winery: Tormaresca
Statistical Match: 17.48%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: 31.0
Wine: Castello Banfi 2009 Cum Laude Red (Toscana)
Province: Tuscany
Variety: Red Blend
Winery: Castello Banfi
Statistical Match: 17.46%
---------------------------------------------------------------------------------
Country:

### Che vino mi consigli di bere con la carne?

In [387]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 2
Enter your wine recommendation query in Italian: Che vino mi consigli di bere con la carne?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: 30.0
Wine: Marchesi Antinori 2010 Guado al Tasso Il Bruciato  (Bolgheri)
Province: Tuscany
Variety: Red Blend
Winery: Marchesi Antinori
Statistical Match: 22.88%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: 26.0
Wine: Renieri 2007 Riserva  (Chianti Classico)
Province: Tuscany
Variety: Sangiovese
Winery: Renieri
Statistical Match: 20.57%
---------------------------------------------------------------------------------
Country: Italy
Score: 92/100
Price: 80.0
Wine: Talenti 2007 Pian di Conte Riserva  (Brunello di Montalcino)
Province: Tuscany
Variety: Sangiovese Grosso
Winery: Talenti
Statistical Match: 19.83%
---------------------------------------------------------------------------

### Quale è il miglior vino da bere con il pesce?

In [383]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 2
Enter your wine recommendation query in Italian: Quale è il miglior vino da bere con il pesce?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: nan
Wine: Bellavista 2002 Vittorio Moretti Extra Brut Sparkling (Franciacorta)
Province: Lombardy
Variety: Sparkling Blend
Winery: Bellavista
Statistical Match: 17.95%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 80.0
Wine: Speri 2008 Vigneto Monte Sant'Urbano  (Amarone della Valpolicella Classico)
Province: Veneto
Variety: Corvina, Rondinella, Molinara
Winery: Speri
Statistical Match: 10.64%
---------------------------------------------------------------------------------
Country: Italy
Score: 93/100
Price: 64.0
Wine: Terrabianca 2004 Ceppate Cabernet Sauvignon (Toscana)
Province: Tuscany
Variety: Cabernet Sauvignon
Winery: Terrabianca
Statistical Match: 9.53%
------------------

### Quale è il miglior vino da bere con i frutti di mare?

In [385]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 2
Enter your wine recommendation query in Italian: Quale è il miglior vino da bere con frutti di mare?


********** Recommended Wines **********

Country: Italy
Score: 91/100
Price: 55.0
Wine: Cantina Terlano 2008 Quarz Sauvignon (Alto Adige)
Province: Northeastern Italy
Variety: Sauvignon
Winery: Cantina Terlano
Statistical Match: 19.52%
---------------------------------------------------------------------------------
Country: Italy
Score: 91/100
Price: nan
Wine: Ronchi di Manzano 1999 Superiore Tocai (Colli Orientali del Friuli)
Province: Northeastern Italy
Variety: Tocai
Winery: Ronchi di Manzano
Statistical Match: 12.84%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 80.0
Wine: Speri 2008 Vigneto Monte Sant'Urbano  (Amarone della Valpolicella Classico)
Province: Veneto
Variety: Corvina, Rondinella, Molinara
Winery: Speri
Statistical Match: 11.01%
---

### Qual è il miglior vino da bere con il formaggio?

In [386]:
recommend_wines()

Select language for input:
1. English
2. Italian
Enter 1 or 2: 2
Enter your wine recommendation query in Italian: Qual è il miglior vino da bere con il formaggio?


********** Recommended Wines **********

Country: Italy
Score: 92/100
Price: 60.0
Wine: Tenuta Monteti 2006 Monteti Red (Toscana)
Province: Tuscany
Variety: Red Blend
Winery: Tenuta Monteti
Statistical Match: 16.38%
---------------------------------------------------------------------------------
Country: Italy
Score: 94/100
Price: 60.0
Wine: Antonio Caggiano 2004 Vigna Macchia dei Goti  (Taurasi)
Province: Southern Italy
Variety: Aglianico
Winery: Antonio Caggiano
Statistical Match: 15.63%
---------------------------------------------------------------------------------
Country: Italy
Score: 95/100
Price: nan
Wine: Falesco 2007 Passirò White (Lazio)
Province: Central Italy
Variety: White Blend
Winery: Falesco
Statistical Match: 15.60%
---------------------------------------------------------------------------------
Country